# Uncertainty Analysis

Explore uncertainty quantification.

In [ ]:
import sys
sys.path.insert(0, '../src')

import numpy as np
import matplotlib.pyplot as plt
import torch
import pandas as pd

from uncertainty_module import UncertaintyOversightEngine
from uncertainty_module.core.engine import RiskLevel

print("✓ Imports")

In [ ]:
# Setup
engine = UncertaintyOversightEngine(mc_dropout_samples=30)

# Simple model
import torch.nn as nn
model = nn.Sequential(
    nn.Flatten(),
    nn.Linear(1024*1024, 128),
    nn.ReLU(),
    nn.Dropout(0.5),
    nn.Linear(128, 14)
)

class_names = ["No Finding", "Pneumonia", "Cardiomegaly", "Edema", "Pleural Effusion",
               "Lung Opacity", "Atelectasis", "Consolidation", "Pneumothorax",
               "Enlarged Cardiomediastinum", "Lung Lesion", "Fracture",
               "Pleural Other", "Support Devices"]

engine.register_model(model, class_names)
print("✓ Engine ready")

In [ ]:
# Generate test images
np.random.seed(42)
images = {
    'normal': [torch.randn(1, 1024, 1024) * 0.3 + 0.5 for _ in range(50)],
    'noisy': [torch.randn(1, 1024, 1024) * 0.6 + 0.5 for _ in range(50)],
}

results = []
for img_type, img_list in images.items():
    for i, img in enumerate(img_list):
        result = engine.analyze(img, f"{img_type}_{i}")
        results.append({
            'type': img_type,
            'total_unc': result.uncertainty.total_uncertainty,
            'epistemic': result.uncertainty.epistemic_uncertainty,
            'confidence': result.uncertainty.confidence_score,
            'risk': result.uncertainty.risk_level.value,
        })

df = pd.DataFrame(results)
df.head()

In [ ]:
# Visualize
fig, axes = plt.subplots(1, 2, figsize=(12, 5))

ax = axes[0]
for img_type in ['normal', 'noisy']:
    ax.hist(df[df['type']==img_type]['total_unc'], bins=20, alpha=0.6, label=img_type)
ax.set_xlabel('Total Uncertainty')
ax.legend()

ax = axes[1]
ax.scatter(df['confidence'], df['total_unc'], c=df['type'].map({'normal': 0, 'noisy': 1}), alpha=0.5)
ax.set_xlabel('Confidence')
ax.set_ylabel('Uncertainty')

plt.tight_layout()
plt.show()